<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/Lake-Monitoring-Sentinel2/blob/main/LakeMonitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install xee -q

In [3]:
import ee
import geemap
import xarray as xr
import xee
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
import matplotlib.colors as mcolors

In [5]:
ee.Authenticate()
ee.Initialize(project="Your google earh engine project name", opt_url='https://earthengine-highvolume.googleapis.com')

In [11]:
# Select the area of intrest
map=geemap.Map(basemap="SATELLITE")
map

In [11]:
roi = map.draw_last_feature.geometry()

start_time = '2015-01-01'
end_time = '2024-12-31'

sentinel = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterBounds(roi).filterDate(start_time,end_time).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)).select((['B3', 'B8']));
def add_year(img):
    date = ee.Date(img.get('system:time_start'))
    year = date.get('year')
    return img.set('year', year)

sentinel_with_year = sentinel.map(add_year)



In [8]:
years = sentinel_with_year.aggregate_array('year').distinct().sort()

def yearly_mean(year):
    year = ee.Number(year)
    filtered = sentinel_with_year.filter(ee.Filter.eq('year', year))
    mean_image = filtered.mean().set('year', year)
    return mean_image

yearly_means = ee.ImageCollection(years.map(yearly_mean))

In [11]:
def ndwi(img):
     index = img.normalizedDifference(['B3', 'B8']).rename('ndwi')
     return index.copyProperties(img, img.propertyNames())

ndwi_yearly = yearly_means.map(ndwi)

In [11]:
years = ndwi_yearly.aggregate_array('year').getInfo()

ds = xr.open_dataset(ndwi_yearly, engine='ee', crs='EPSG:4326', scale=0.00009, geometry=roi)

new_time = pd.to_datetime([str(y) for y in years]).year
ds['time'] = new_time

ndwi_annual = ds['ndwi']

g = ndwi_annual.plot(
    x='lon',
    y='lat',
    cmap='Blues',
    col='time',
    col_wrap=5,
    robust=True
)

for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))


plt.savefig('lake_desiccation.png', dpi=360, bbox_inches='tight')


In [11]:
df = ds.to_dataframe().reset_index()

df = df.dropna(subset=['ndwi'])

model = KMeans(n_clusters=2, random_state=42)
df['clustering'] = model.fit_predict(df[['ndwi']])

xarr = df.set_index(['time', 'lat', 'lon']).to_xarray()

xarr = xarr.sortby('lon').sortby('lat')


lake_pixels_per_year = (xarr.clustering == 1).sum(dim=['lat', 'lon'])

lake_area_km2 = lake_pixels_per_year * 100 / 1_000_000  # each pixel = 900 m²
lake_area_values = lake_area_km2.values

years = xarr['time'].values
n_years = len(years)
n_cols = 5
n_rows = (n_years + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4 * n_rows), constrained_layout=True)
axes = axes.flatten()

for i, year in enumerate(years):
    data = xarr.clustering.sel(time=year)
    im = data.plot(
        ax=axes[i],
        x='lon',
        y='lat',
        cmap='Blues',
        add_colorbar=False
    )
    year_str = pd.to_datetime(str(year)).year
    area = lake_area_values[i]
    axes[i].set_title(f"{year_str}")
    axes[i].text(
        0.05, 0.05,
        f"Lake area: {area:.1f} km²",
        transform=axes[i].transAxes,
        fontsize=10,
        bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3')
    )

for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.savefig("lake_clustering_with_area_labels.png", dpi=360, bbox_inches='tight')
plt.show()